In [121]:
from qiskit_optimization import QuadraticProgram
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms.utils import algorithm_globals

In [122]:
mod = QuadraticProgram("Optimization_Problem")
print("Number of variables:", len(mod.variables))



Number of variables: 0


In [123]:
N=5
Dist=2 # Distance to travel

tolerance = 2 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1
delta_v_sq=delta_v**2

vmax=150 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

# Define two lists of binary variables
x = mod.binary_var_list(N, name="x")
y = mod.binary_var_list(N, name="y")
z = mod.binary_var_list(N, name="z")
print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  0

Subject to
  No constraints

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14

Number of variables: 15


### Defining objective function

In [124]:
linear_terms = {var.name: delta_v_sq for var in x}
linear_terms.update({var.name: -alpha*delta_v_sq for var in y})
mod.minimize(linear=linear_terms)
print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  No constraints

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14

Number of variables: 15


### Adding constraints

#### Net-Zero constraint

In [125]:
constraint_terms = {var.name: 1 for var in x}
constraint_terms.update({var.name: -1 for var in y})
mod.linear_constraint(constraint_terms, '=', 0, name="net-zero_constraint")
print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (1)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14

Number of variables: 15


#### Maximum Speed Constraint

In [126]:
constraint_terms = {var.name: delta_v for var in x}
mod.linear_constraint(constraint_terms, '<=', vmax, name="Maximum_Speed_constraint")
print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (2)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14

Number of variables: 15


#### Positive speed constraint

In [127]:
for i in range(N):
    constraint_terms = {var.name: 1 for var in x[0:i]}
    constraint_terms.update({var.name: -1 for var in y[0:i]})
    mod.linear_constraint(constraint_terms, '>=', 0, name="Positive_Speed"+str(i))

print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (7)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'
    0 >= 0  'Positive_Speed0'
    x0 - y5 >= 0  'Positive_Speed1'
    x0 + x1 - y5 - y6 >= 0  'Positive_Speed2'
    x0 + x1 + x2 - y5 - y6 - y7 >= 0  'Positive_Speed3'
    x0 + x1 + x2 + x3 - y5 - y6 - y7 - y8 >= 0  'Positive_Speed4'

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14

Number of variables: 15


#### No simultaneous acceleration/deceleration

In [128]:


# Constraint 1: z[i] <= x[i]
for i in range(N):
    mod.linear_constraint({x[i].name: 1, z[i].name: -1}, '<=', 0, name=f"z_u_d_{i}")

# Constraint 2: z[i] <= y[i]
for i in range(N):
    mod.linear_constraint({y[i].name: 1, z[i].name: -1}, '<=', 0, name=f"z_p_d_{i}")

# Constraint 3: z[i] >= x[i] + y[i] - 1
for i in range(N):
    mod.linear_constraint({x[i].name: 1, y[i].name: 1, z[i].name: -1}, '>=', -1, name=f"z_u_p_d_{i}")

# Final Constraint: sum(z[i]) == 0 (No simultaneous braking or acceleration)
sum_constraint = {z[i].name: 1 for i in range(N)}
mod.linear_constraint(sum_constraint, '==', 0, name="No_simultaneous_braking_or_acceleration_constraint")

# Print the QuadraticProgram to see the problem
print(mod.prettyprint())
print("Number of variables:", len(mod.variables))

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (23)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'
    0 >= 0  'Positive_Speed0'
    x0 - y5 >= 0  'Positive_Speed1'
    x0 + x1 - y5 - y6 >= 0  'Positive_Speed2'
    x0 + x1 + x2 - y5 - y6 - y7 >= 0  'Positive_Speed3'
    x0 + x1 + x2 + x3 - y5 - y6 - y7 - y8 >= 0  'Positive_Speed4'
    x0 - z10 <= 0  'z_u_d_0'
    x1 - z11 <= 0  'z_u_d_1'
    x2 - z12 <= 0  'z_u_d_2'
    x3 - z13 <= 0  'z_u_d_3'
    x4 - z14 <= 0  'z_u_d_4'
    y5 - z10 <= 0  'z_p_d_0'
    y6 - z11 <= 0  'z_p_d_1'
    y7 - z12 <= 0  'z_p_d_2'
    y8 - z13 <= 0  'z_p_d_3'
    y9 - z14 <= 0  'z_p_d_4'
    x0 + y5 - z10 >= -1  'z_u_p_d_0'
    x1 + y6 - z11 >= -1  'z_u_p_d_1'
    x2 + y7 - z12 >= -1  'z_u_p_d_2'
    x3 + y8 - z13 >= -1  'z_u_p_d_3'
    x4 + y9 - z14 >= -1  'z_

#### Distance constraint

In [129]:
# Define the linear expression for the total distance
linear_terms = {}
for i in range(N):
    # Each term in the summation contributes delta_v * (x[i] - y[i])
    linear_terms[x[i].name] = delta_v  # x[i] contributes +delta_v
    linear_terms[y[i].name] = -delta_v  # y[i] contributes -delta_v

# Add constraints for distance (distance <= Dist + tolerance) and (distance >= Dist - tolerance)
mod.linear_constraint(linear_terms, '<=', Dist + tolerance, name="Max_Distance_constraint")
mod.linear_constraint(linear_terms, '>=', Dist - tolerance, name="Min_Distance_constraint")

# Print the QuadraticProgram to see the problem
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (25)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'
    0 >= 0  'Positive_Speed0'
    x0 - y5 >= 0  'Positive_Speed1'
    x0 + x1 - y5 - y6 >= 0  'Positive_Speed2'
    x0 + x1 + x2 - y5 - y6 - y7 >= 0  'Positive_Speed3'
    x0 + x1 + x2 + x3 - y5 - y6 - y7 - y8 >= 0  'Positive_Speed4'
    x0 - z10 <= 0  'z_u_d_0'
    x1 - z11 <= 0  'z_u_d_1'
    x2 - z12 <= 0  'z_u_d_2'
    x3 - z13 <= 0  'z_u_d_3'
    x4 - z14 <= 0  'z_u_d_4'
    y5 - z10 <= 0  'z_p_d_0'
    y6 - z11 <= 0  'z_p_d_1'
    y7 - z12 <= 0  'z_p_d_2'
    y8 - z13 <= 0  'z_p_d_3'
    y9 - z14 <= 0  'z_p_d_4'
    x0 + y5 - z10 >= -1  'z_u_p_d_0'
    x1 + y6 - z11 >= -1  'z_u_p_d_1'
    x2 + y7 - z12 >= -1  'z_u_p_d_2'
    x3 + y8 - z13 >= -1  'z_u_p_d_3'
    x4 + y9 - z14 >= -1  'z_

### Convert optimization problem

In [130]:
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(mod)
print("Number of variables:", len(qubo.variables))

Number of variables: 47


Our optimization problem is now defined, let's solve it!

### Get backend

In [131]:
# Load saved credentials
service = QiskitRuntimeService()
# Get the least busy backend
backend = service.least_busy(operational=True, simulator=False)
print("least busy backend: ", backend)

least busy backend:  <IBMBackend('ibm_kyiv')>


In [133]:
qaoa = QAOA(sampler=Sampler(mode=backend), optimizer=COBYLA())
optimizer = MinimumEigenOptimizer(qaoa)  # using QAOA

print(optimizer.is_compatible(problem=qubo))
print(optimizer.get_compatibility_msg(problem=qubo))

True



In [134]:
qaoa_result = optimizer.solve(qubo)
print(qaoa_result.prettyprint())

TypeError: SamplerV2.run() takes 2 positional arguments but 3 were given